# Vectorstore Agent

This notebook showcases an agent designed to retrieve information from one or more vectorstores, either with or without sources.

## Create the Vectorstores

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
llm = OpenAI(temperature=0)

In [2]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
state_of_union_store = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [3]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_store = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


## Initalize Toolkit and Agent

First, we'll create an agent with a single vectorstore.

In [4]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
)
toolkit = VectorStoreToolkit(
    name="state_of_union_address",
    description="Information about the most recent state of the Union "
    vectorstore=state_of_union_store
)
agent_executor = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

ValidationError: 1 validation error for VectorStoreToolkit
description
  field required (type=value_error.missing)

## Examples

In [8]:
agent_executor.run("What did biden say about ketanji brown jackson is the state of the union address?")



> Entering new AgentExecutor chain...
 I need to find the answer in the state of the union address
Action: state_of_union_address
Action Input: What did biden say about ketanji brown jackson
Observation:  Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.
Thought: I now know the final answer
Final Answer: Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.

> Finished chain.


"Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence."

In [9]:
agent_executor.run("What did biden say about ketanji brown jackson is the state of the union address? List the source.")



> Entering new AgentExecutor chain...
 I need to use the state_of_union_address_with_sources tool to answer this question.
Action: state_of_union_address_with_sources
Action Input: What did biden say about ketanji brown jackson
Observation: {"answer": " Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.\n", "sources": "../../state_of_the_union.txt"}
Thought: I now know the final answer
Final Answer: Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. Source: ../../state_of_the_union.txt

> Finished chain.


"Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. Source: ../../state_of_the_union.txt"

## Multiple Vectorstores
We can also easily use this initialize an agent with multiple vectorstores. To do this, we can create another toolkit and then just combine the tools.

In [9]:

sota_toolkit = VectorStoreToolkit(
    vectorstores=[
        NamedVectorStore(
            name="state_of_union_address",
            vectorstore=state_of_union_store
        ),
        NamedVectorStore(
            name="ruff_python_linter",
            vectorstore=ruff_store
        ),
    ],
    llm=llm
)


Exiting: Cleaning up .chroma directory
Exiting: Cleaning up .chroma directory


## Examples

In [10]:
agent_executor.run("What did biden say about ketanji brown jackson is the state of the union address?")



> Entering new AgentExecutor chain...
 I need to find the answer in the state of the union address
Action: state_of_union_address
Action Input: What did biden say about ketanji brown jackson
Observation:  Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.
Thought: I now know the final answer
Final Answer: Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.

> Finished chain.


"Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence."

In [11]:
agent_executor.run("What did biden say about ketanji brown jackson is the state of the union address? List the source.")



> Entering new AgentExecutor chain...
 I need to use the state_of_union_address_with_sources tool to answer this question.
Action: state_of_union_address_with_sources
Action Input: What did biden say about ketanji brown jackson
Observation: {"answer": " Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, one of the nation's top legal minds, to continue Justice Breyer's legacy of excellence.\n", "sources": "../../state_of_the_union.txt"}
Thought: I now know the final answer
Final Answer: Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, one of the nation's top legal minds, to continue Justice Breyer's legacy of excellence. Source: ../../state_of_the_union.txt

> Finished chain.


"Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, one of the nation's top legal minds, to continue Justice Breyer's legacy of excellence. Source: ../../state_of_the_union.txt"

In [14]:
agent_executor.run("Why use ruff over flake8? List the source.")



> Entering new AgentExecutor chain...
 I need to use the ruff_python_linter_with_sources tool to answer this question.
Action: ruff_python_linter_with_sources
Action Input: Why use ruff over flake8?
Observation: {"answer": " Ruff is a linter that can be used as a drop-in replacement for Flake8 when used with certain conditions. It implements a subset of the E and W rules from pycodestyle, and re-implements some of the most popular Flake8 plugins and related code quality tools natively.\n", "sources": "https://beta.ruff.rs/docs/faq/"}
Thought: I now know the final answer
Final Answer: Ruff is a linter that can be used as a drop-in replacement for Flake8 when used with certain conditions. It implements a subset of the E and W rules from pycodestyle, and re-implements some of the most popular Flake8 plugins and related code quality tools natively. Sources: https://beta.ruff.rs/docs/faq/

> Finished chain.


'Ruff is a linter that can be used as a drop-in replacement for Flake8 when used with certain conditions. It implements a subset of the E and W rules from pycodestyle, and re-implements some of the most popular Flake8 plugins and related code quality tools natively. Sources: https://beta.ruff.rs/docs/faq/'

In [15]:
agent_executor.run("What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?")



> Entering new AgentExecutor chain...
 I need to answer a question about ruff and the state of the union.
Action: ruff_python_linter
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff is integrated into nbQA, a tool for running linters and code formatters over Jupyter Notebooks. After installing ruff and nbqa, you can run Ruff over a notebook like so: > nbqa ruff Untitled.ipynb
Thought: I need to check if the president mentioned this tool in the state of the union
Action: state_of_union_address
Action Input: Did the president mention nbQA in the state of the union?
Observation:  No, the president did not mention nbQA in the state of the union.
Thought: I now know the final answer
Final Answer: No, the president did not mention nbQA in the state of the union.

> Finished chain.


'No, the president did not mention nbQA in the state of the union.'